In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD
from typing import *
from tqdm.notebook import tqdm
import string
import numpy as np
import json
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
!pip install pytorch-lightning
import pytorch_lightning as pl
!pip install transformers
import transformers
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from transformers import BertForSequenceClassification, AdamW
from pytorch_lightning.loggers import TensorBoardLogger
TRAIN = False

     |████████████████████████████████| 525 kB 10.4 MB/s 
     |████████████████████████████████| 132 kB 52.8 MB/s 
     |████████████████████████████████| 332 kB 34.7 MB/s 
     |████████████████████████████████| 829 kB 37.0 MB/s 
     |████████████████████████████████| 596 kB 38.8 MB/s 
     |████████████████████████████████| 1.1 MB 48.4 MB/s 
     |████████████████████████████████| 160 kB 54.4 MB/s 
     |████████████████████████████████| 271 kB 52.3 MB/s 
     |████████████████████████████████| 192 kB 53.4 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=d38e85042ac0bb40fbb8dba69c6c9ea1649ead382fe935a4ba7bcf065649b479
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found exi

In [ ]:
laptop_train = 'drive/My Drive/nlp2021-hw2/data/laptops_train.json'
laptop_dev = 'drive/My Drive/nlp2021-hw2/data/laptops_dev.json'
restaurants_train = 'drive/My Drive/nlp2021-hw2/data/restaurants_train.json'
restaurants_dev = 'drive/My Drive/nlp2021-hw2/data/restaurants_dev.json'

data = {}
f = open(laptop_train,)
data[laptop_train] = json.load(f)  
f = open(laptop_dev,)
data[laptop_dev] = json.load(f) 
f = open(restaurants_train,)
data[restaurants_train] = json.load(f) 
f = open(restaurants_dev,)
data[restaurants_dev] = json.load(f) 

In [ ]:
from transformers import BertTokenizerFast, BertConfig


class Preprocesser_b():

  def __init__(self):

    self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased', do_lower_case=False)
    self.tag_2_idx = {'[PAD]':0, 'O': 1, 'B': 2, 'I': 3}
    self.sent_2_idx = {'positive': 0, 'negative': 1, 'neutral': 2, 'conflict': 3}
   
  def preprocess_for_b(self, data):
    self.samples = {}
    self.indexed_samples = []
    sentence_counter = 0
    for entry in data:
      entry['tokenized'] = self.prepare_sentence(entry['text'])
      inputs = self.tokenizer(entry['text'], return_offsets_mapping = True)
      entry['input_ids'] = inputs["input_ids"]
      entry['attention_mask'] = inputs['attention_mask']
      entry['label'] = ['O' for token in entry['input_ids']]
      entry['maps'] = inputs['offset_mapping']
      entry['tokenized'].insert(0, '[CLS]')
      entry['tokenized'].append('[SEP]')
      counter = 0
      for token in entry['tokenized']:
        if token == '[CLS]' or token == '[SEP]':
          counter += 1
          continue
        for target in entry['targets']:
          start = int(target[0][0])
          end = int(target[0][1])
          if entry['maps'][counter][0] == start:
            entry['label'][counter] = 'B'
          elif entry['maps'][counter][0] > start and entry['maps'][counter][1] < end:
            entry['label'][counter] = 'I'
          elif entry['maps'][counter][1] == end:
              entry['label'][counter] = 'I'
        counter += 1     
      entry = self.create_window(entry, window_size=512) 
      entry['sentiments'] = self.extract_sentiments(entry)
      entry['sentence_counter'] = sentence_counter
      sentence_counter += 1
    targets_dataset = self.expand(data)
    return targets_dataset

  def expand(self, data):
    '''
    Creates a dataset where each samples are single aspects, and not  sentences. Returns said dataset and a mapping, connecting samples of this dataset to samples of the input one
    '''

    target_dataset = []
    new_sample_counter = 0
    for entry in  data:
      for tgt_idx in range(0, len(entry['sentiments'])):
        if entry['sentiments'] != []:
          inputs = entry['input_ids'] + entry['aspects'][tgt_idx] + [102]
          attention = entry['attention_mask'] + [1]*(len(entry['aspects'][tgt_idx]) + 1)  #add 1's to attention mask equal to number of tokens added to input (+1 for final sep token)       
          new_sample = {'new_sample_counter':new_sample_counter, 
                        'target': entry['sentiments'][tgt_idx],
                        'input_ids': inputs,
                        'attention_mask':attention,
                        'to_be_mapped_to': entry['sentence_counter']}
          new_sample['attention_mask'] = self.pad(new_sample['attention_mask'], 512)
          new_sample['input_ids'] = self.pad(new_sample['input_ids'], 512)
          target_dataset.append(new_sample)
          new_sample_counter +=1
    return target_dataset
  
  


  def create_window(self, entry, window_size):

    window = entry["input_ids"]
    actual_length = len(window)
    if len(window) < window_size:
      window = window + ['[PAD]']*(window_size - len(window))
      entry["label"] = entry["label"] + ['[PAD]']*(window_size - len(entry['label']))
      assert len(window) == window_size
    for i in range (0, len(entry['label'])):
      entry['label'][i] = self.tag_2_idx[entry['label'][i]]
    entry['label'] = torch.tensor(entry['label'])
    entry['label'] = entry['label'].to(device)
    entry['indexes'] = self.extract_aspects(entry) 
    entry['aspects'] = [[entry['input_ids'][index] for index in aspect] for aspect in entry['indexes']]
    return entry

    
  def pad(self, to_be_padded, size):

    padded = to_be_padded + [0]*(size-len(to_be_padded))
    return torch.tensor(padded)


  def prepare_sentence(self, sent):
    tokenized = self.tokenizer.tokenize(sent)
    return tokenized

  def extract_aspects(self, sample):
    block = (0,0)
    blocks = []
    for idx in range (0, len(sample['label'])):
      if sample['label'][idx] == 1:
        start = 0
        end = 0
        continue
      elif sample['label'][idx] == 2:
        start = idx
        end = idx
      elif sample['label'][idx] == 3:
        if sample['label'][idx - 1] in [2, 3]:
          end = idx
        elif sample['label'][idx - 1] == 1:
          start = idx
          end = idx
      if sample['label'][idx + 1] in [1,2] and sample['label'][idx] != 1:
        blocks.append((start, end))
        start = 0
        end = 0
      if sample['label'][idx + 1] == 0:
        indexes = [[number for number in range(lista[0], lista[1] + 1)] for lista in blocks]
        return indexes

  def extract_sentiments(self, sample):
    sentiments = [self.sent_2_idx[target[2]] for target in sample['targets']]


    return sentiments



#[[[55, 63], 'keyboard', 'positive'], [[73, 78], 'speed', 'positive']]

    

    
        

In [ ]:
class ReviewDataset(Dataset):

  def __init__(self, data):
    self.data = data
    self.setup()

  def setup(self):
    self.preprocesser = Preprocesser_b()
    self.data = self.preprocesser.preprocess_for_b(self.data)
  
  def __len__(self):
    return len(self.data)

  def convert_to_str(self, stuff):
    stringa = ';'.join([str(item) for item in stuff])
    return stringa

  def __getitem__(self, index: int):
    data_row = self.data[index]
    return dict(
      input_ids = data_row['input_ids'],
      attention_mask=data_row["attention_mask"],
      labels=data_row['target'],
      map = data_row['to_be_mapped_to'],
      number=data_row['new_sample_counter']
      )


In [ ]:
BATCH_SIZE = 8
class ABSADataModule(pl.LightningDataModule):
    def __init__(self, training_file1, training_file2, dev_file1, dev_file2, window_size, window_shift, label_vocabulary=['B', 'I', 'O']):
        super().__init__()
        self.training_file1 = training_file1
        self.training_file2 = training_file2
        self.dev_file1 = dev_file1
        self.dev_file2 = dev_file2
        self.prepare_data()


    def prepare_data(self):
      self.preprocesser = Preprocesser_b()
      f = open(self.training_file1,)
      temp = json.load(f)
      f = open(self.training_file2,)
      temp2 = json.load(f) 
      train_data = [*temp, *temp2]
      self.training_data = ReviewDataset(train_data)
      f = open(self.dev_file1,)
      temp = json.load(f)
      f = open(self.dev_file2,)
      temp2 = json.load(f) 
      dev_data = [*temp, *temp2]
      self.dev_data = ReviewDataset(dev_data)    
          
    def train_dataloader(self):
      return DataLoader(self.training_data, batch_size=BATCH_SIZE)
    
    def val_dataloader(self):
        return DataLoader(self.dev_data, batch_size=BATCH_SIZE)

In [ ]:
num_labels = 4
class ModelA(pl.LightningModule):
  def __init__(self,num_labels):
    super().__init__()
    self.bert = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=num_labels, return_dict=True)
    self.loss_fn = torch.nn.CrossEntropyLoss()
    self.loss_fn2 = torch.nn.BCELoss()
    #self.loss_fn2 = SelfAdjDiceLoss()
    self.batch_size = BATCH_SIZE
    #self.val_f1 = pl.metrics.F1(num_labels)
    #self.test_accuracy = pl.metrics.F1(num_labels)

  def forward(self, input_ids, attention_mask, y=None):
    output = self.bert(input_ids, attention_mask)
    logits = torch.swapaxes(output.logits, 1, 2)
    loss = 0
    preds = torch.argmax(logits, dim=1)
    if y is not None:
        loss = self.loss(logits, y)
    return loss, logits

  def loss(self, predictions, golden):
    return self.loss_fn(predictions, golden)

  def loss2(self, predictions, golden):
    return self.loss_fn2(predictions, golden)

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self.forward(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True, batch_size = self.batch_size)
    return {"loss": loss, "predictions": outputs, "labels": labels}
  
  def validation_step(self, batch, batch_idx):
    with torch.no_grad():
      input_ids = torch.tensor(batch["input_ids"])
      attention_mask = batch["attention_mask"]
      labels = batch["labels"]
      loss, outputs = self(input_ids, attention_mask, labels)
      self.log("val_loss", loss, prog_bar=True, logger=True, batch_size = self.batch_size)
    return loss

  def configure_optimizers(self):
      optimizer = torch.optim.SGD(self.parameters(), lr=0.05, momentum=0.0)
      return optimizer


In [ ]:
#directory = '/content/drive/MyDrive/nlp2021-hw2/model/model.pt'
#trained_model = torch.load(directory)

In [ ]:
data_module = ABSADataModule(laptop_train, restaurants_train, laptop_dev, restaurants_dev, window_size = 3, window_shift = 1)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
a = data_module.training_data[0]
a

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0

In [ ]:
#@title
import ast
def convert_label(entry, labels, flag):
  #Takes as input a sample and its label (predicted or golden); converts label in the format of {'target_word_1': [start, end], 'target_word_2': [start2, end2]}
  tokens = ''
  aspects = []
  start = 0
  end = 0
  if flag == 'lab':
    labels = entry['labels']
  token_list = entry['tokens'].split(';')
  length = len(token_list)
  map = entry['map'].split(';')
  for i in range(0, len(map)):
    map[i] = ast.literal_eval(map[i])
  for idx in range(0, length):
    considered = token_list[idx]
    if labels[idx] == 2:
      tokens = considered
      start = map[idx][0]   
      if labels[idx + 1] == 3:
        continue
      else:
        end = map[idx][1]
        dictionary = {tokens: (start, end)}
        aspects.append(dictionary)
        start = 0
        end = 0
        tokens = ''
    elif labels[idx] == 3:
      if considered.startswith('##'):
        if tokens == '': 
          tokens = tokens +  token_list[idx - 1] + considered.strip('##')
        else:
          tokens = tokens +  considered.strip('##')
      else:
        if tokens == '':
          start = map[idx][0]
          tokens = tokens + considered
        else:
          tokens = tokens + ' ' + considered
        
      if labels[idx + 1] == 3:
        continue
      else:
        end = map[idx][1]
        dictionary = {tokens: (start, end)}
        aspects.append(dictionary)
        start = 0
        end = 0
        tokens = ''
  return aspects

In [ ]:
num_labels = 4
class ModelB(pl.LightningModule):
  def __init__(self,num_labels):
    super().__init__()
    self.bert = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=num_labels, return_dict=True)
    self.loss_fn = torch.nn.CrossEntropyLoss()
    self.batch_size = BATCH_SIZE

  def forward(self, input_ids, attention_mask, y=None):
    output = self.bert(input_ids, attention_mask)
    logits = output.logits
    #batch_size, sequence_length, hidden_size
    loss = 0

    if y is not None:
        loss = self.loss(logits, y)
    return loss, logits

  def loss(self, predictions, golden):
    return self.loss_fn(predictions, golden)


  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self.forward(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True, batch_size = self.batch_size)
    return {"loss": loss, "predictions": outputs, "labels": labels}
  
  def validation_step(self, batch, batch_idx):
    with torch.no_grad():
      input_ids = torch.tensor(batch["input_ids"])
      attention_mask = batch["attention_mask"]
      labels = batch["labels"]
      loss, outputs = self(input_ids, attention_mask, labels)
      self.log("val_loss", loss, prog_bar=True, logger=True, batch_size = self.batch_size)
    return loss

  def configure_optimizers(self):
      optimizer = torch.optim.SGD(self.parameters(), lr=0.0005, momentum=0.0)
      return optimizer
            

In [ ]:
#@title
N_EPOCHS = 10
checkpoint_callback = ModelCheckpoint(dirpath="drive/My Drive/nlp2021-hw2/",filename="best_model", save_top_k=1, verbose=True, monitor="val_loss",mode="min")
logger = TensorBoardLogger("lightning_logs", name="absa")
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)
trainer = pl.Trainer(max_epochs=N_EPOCHS,gpus=1,logger=logger,callbacks=[early_stopping_callback],checkpoint_callback=checkpoint_callback)
model = ModelB(num_labels)
model.cuda()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f8c65734610>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f8c65734610>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

ModelB(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
     

In [ ]:
if TRAIN == True:
  trainer.fit(model, data_module)
  trained_model = ModelB.load_from_checkpoint(trainer.checkpoint_callback.best_model_path,num_labels=num_labels)
else:
  directory = '/content/drive/MyDrive/nlp2021-hw2/model/model_049_3.pt'
  trained_model = torch.load(directory)

In [ ]:
directory = '/content/drive/MyDrive/nlp2021-hw2/model/model_049_3.pt'
#torch.save(trained_model, directory)

trained_model = torch.load(directory)


In [ ]:
type(data_module.dev_data)

__main__.ReviewDataset

In [ ]:
preds = []
labs = []
with torch.no_grad():
  trained_model = trained_model.to(device)
  predictions = []
  labels = []
  right = 0
  total = 0
  for item in tqdm(data_module.dev_data):
      inputs = item["input_ids"].unsqueeze(dim=0).to(device)
      mask = item["attention_mask"].unsqueeze(dim=0).to(device)
      prediction = trained_model.bert(inputs, mask)
      prediction = torch.argmax(prediction.logits, dim=1)
      #prediction = torch.argmax(prediction.squeeze(dim=0), dim=0)
      sentence_number = item['map']
      if prediction == item['labels']:
        right += 1
      predictions.append(prediction)
      labels.append(torch.tensor(item["labels"]))
      total += 1
      print(right, total)

  predictions = torch.stack(predictions).detach().cpu()
  labels = torch.stack(labels).detach().cpu()

  0%|          | 0/1094 [00:00<?, ?it/s]

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
9 11
9 12
9 13
9 14
9 15
10 16
11 17
12 18
12 19
13 20
14 21
14 22
14 23
14 24
15 25
16 26
17 27
18 28
19 29
19 30
20 31
20 32
20 33
20 34
21 35
21 36
22 37
23 38
24 39
24 40
25 41
26 42
27 43
28 44
29 45
29 46
29 47
30 48
30 49
30 50
30 51
31 52
31 53
32 54
33 55
34 56
35 57
35 58
36 59
36 60
37 61
37 62
38 63
39 64
40 65
41 66
42 67
43 68
44 69
45 70
45 71
46 72
46 73
46 74
47 75
48 76
48 77
49 78
50 79
51 80
52 81
53 82
54 83
55 84
56 85
57 86
58 87
59 88
60 89
60 90
61 91
62 92
63 93
63 94
63 95
64 96
65 97
66 98
67 99
68 100
69 101
70 102
71 103
72 104
73 105
74 106
75 107
75 108
76 109
77 110
78 111
79 112
80 113
80 114
80 115
81 116
82 117
83 118
84 119
85 120
86 121
86 122
87 123
88 124
88 125
89 126
90 127
91 128
92 129
93 130
94 131
95 132
96 133
97 134
98 135
98 136
98 137
98 138
98 139
99 140
100 141
101 142
102 143
103 144
104 145
105 146
105 147
106 148
107 149
108 150
109 151
110 152
110 153
110 154
110 155
111 156
112 157
113 158

In [ ]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
preds = predictions.numpy().squeeze()
labs = labels.numpy()
print(preds, labs)
m = MultiLabelBinarizer(classes=[0,1,2,3])
m.fit(labs)


[2 0 0 ... 0 0 0] [1 0 0 ... 0 0 0]


MultiLabelBinarizer(classes=[0, 1, 2, 3])

In [ ]:
print(f1_score(labs,preds,average='micro'))
print(f1_score(labs,preds,average='macro'))
print(f1_score(labs,preds,average='weighted'))
print(f1_score(labs,preds,average=None))

0.696526508226691
0.493183109350725
0.6722256651256194
[0.79814385 0.63179074 0.48219178 0.06060606]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(labs,preds))

              precision    recall  f1-score   support

           0       0.69      0.95      0.80       546
           1       0.83      0.51      0.63       307
           2       0.59      0.41      0.48       216
           3       0.12      0.04      0.06        25

    accuracy                           0.70      1094
   macro avg       0.56      0.48      0.49      1094
weighted avg       0.70      0.70      0.67      1094



In [ ]:
directory = '/content/drive/MyDrive/nlp2021-hw2/model/model_049_3.pt'
#torch.save(trained_model, directory)

In [ ]:
tgt_2_idx = {'positive': 0, 'negative': 1, 'neutral': 2, 'conflict': 3}
idx_2_tgt = {0: 'positive', 1: 'negative', 2: 'neutral', 3: 'conflict'}

In [ ]:
preprocesser = Preprocesser_b()
a = preprocesser.preprocess_for_b((data[restaurants_dev]))
a[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0

In [ ]:
tok = BertTokenizerFast.from_pretrained('bert-base-cased', do_lower_case=False)
def predict(samples):
  right = 0
  total = 0
  predictions = []
  labels = []
  preds = []
  labs = []
  preprocesser = Preprocesser_b()
  data = preprocesser.preprocess_for_b(samples)
  pred_list = [[] for x in range(0, len(samples))]
  current_sentence = 1
  current_dict = {"targets" : []}
  for item in tqdm(data):
      inputs = item["input_ids"].unsqueeze(dim=0).to(device)
      mask = item["attention_mask"].unsqueeze(dim=0).to(device)
      prediction = trained_model.bert(inputs, mask)
      prediction = torch.argmax(prediction.logits, dim=1)
      #prediction = torch.argmax(prediction.squeeze(dim=0), dim=0)
      if prediction == item['target']:
        right += 1
      indices = [i for i, x in enumerate(list(item['input_ids'])) if x == 102]
      tokens = item['input_ids'][indices[0] + 1:indices[1]]
      aspect = ''
      for token in tokens:
        word_piece = tok.convert_ids_to_tokens([token])[0]
        if "#" in word_piece:
          aspect += word_piece.strip("#")
        elif aspect == '':
          aspect += word_piece
        else:
          aspect += ' ' + word_piece
      pred_list[item['to_be_mapped_to']].append((aspect, idx_2_tgt[int(prediction)]))

      #print(pred_list[item['to_be_mapped_to']])
      predictions.append(prediction)
      labels.append(torch.tensor(item["target"]))
      total += 1
      #print(right, total)
  return pred_list    

  predictions = torch.stack(predictions).detach().cpu()
  labels = torch.stack(labels).detach().cpu()

pred_list = predict(data[restaurants_dev])
print(pred_list)



  0%|          | 0/663 [00:00<?, ?it/s]

[[], [('wines by the glass', 'negative')], [('staff', 'positive')], [], [('Appetizers', 'neutral'), ('main dishes', 'neutral')], [('view', 'positive')], [('reservation', 'negative')], [('half price sushi deal', 'positive')], [], [], [], [], [], [('food', 'positive')], [], [('oil', 'neutral')], [('food', 'positive')], [('staff', 'positive'), ('food', 'positive'), ('place', 'positive')], [], [('dinner', 'neutral')], [('food', 'positive')], [('plate', 'neutral')], [], [], [('desserts', 'positive')], [], [('sushi bar', 'neutral'), ('chef', 'neutral'), ('ala carte', 'neutral')], [('spot', 'positive')], [], [], [], [('pizza', 'positive'), ('santa fe chopped salad', 'positive'), ('fish and chips', 'positive')], [('drinks', 'neutral')], [('lava cake dessert', 'positive')], [], [('seafood dishes', 'neutral'), ('appetizers', 'neutral')], [], [], [('calzones', 'negative')], [('waiters', 'positive')], [], [('glass of wine', 'neutral')], [('pizza', 'positive')], [('drinks', 'positive'), ('wine sele

In [ ]:
for i in range(0, len(data[restaurants_dev])):
  print(data[restaurants_dev][i]['targets'], pred_list[i])

[] []
[[[11, 29], 'wines by the glass', 'negative']] [('wines by the glass', 'negative')]
[[[89, 94], 'staff', 'conflict']] [('staff', 'positive')]
[] []
[[[0, 10], 'Appetizers', 'neutral'], [[48, 59], 'main dishes', 'neutral']] [('Appetizers', 'neutral'), ('main dishes', 'neutral')]
[[[80, 84], 'view', 'positive']] [('view', 'positive')]
[[[11, 22], 'reservation', 'negative']] [('reservation', 'negative')]
[[[26, 47], 'half price sushi deal', 'positive']] [('half price sushi deal', 'positive')]
[] []
[] []
[] []
[] []
[] []
[[[4, 8], 'food', 'positive']] [('food', 'positive')]
[] []
[[[24, 27], 'oil', 'negative']] [('oil', 'neutral')]
[[[4, 8], 'food', 'positive']] [('food', 'positive')]
[[[4, 9], 'staff', 'positive'], [[32, 36], 'food', 'positive'], [[70, 75], 'place', 'positive']] [('staff', 'positive'), ('food', 'positive'), ('place', 'positive')]
[] []
[[[22, 28], 'dinner', 'neutral']] [('dinner', 'neutral')]
[[[5, 9], 'food', 'positive']] [('food', 'positive')]
[[[43, 48], 'plate

In [ ]:
prova = [[]]
prova

[[]]